# Shape detection

In order to perform shape detection, we’ll be using contour approximation.

**Contour approximation is an algorithm for reducing the number of points in a curve with a reduced set of points — thus the term approximation.**

This algorithm is **commonly known as the Ramer-Douglas-Peucker algorithm**, or simply the split-and-merge algorithm.

- Contour approximation **is predicated on the assumption that a curve can be approximated by a series of short line segments.**

 - This leads to a resulting approximated curve that consists of a subset of points that were defined by the original curve.

Contour approximation is actually already implemented in OpenCV via the cv2.approxPolyDP method.

In [1]:
import cv2
import imutils

In [2]:
class ShapeDetector:
    def __init__(self):
        pass
    
    def detect(self, c):
        #Initialize the shape name and approx contour
        shape = "unidentified"
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.04*peri, True)

        #define the shapes

        #triangle
        if len(approx) == 3:
            shape = 'Triangle'

        #Square and rectangle

        elif len(approx)== 4:
            #compute the bounding box of the contour and use the bounding box
            #to compute the aspect ratio
            (x, y, w ,h) = cv2.boundingRect(approx)
            ar = w/float(h)

            #A square has an aspect ratio of 1. Otherwise is a rectangle
            shape = "square" if ar >=0.95 and ar <= 1.05 else "rectangle"
        
        #Pentagon
        elif len(approx) == 5:
            shape = "pentagon"

        #Circle
        else:
            shape = 'circle'
        
        return shape

### Read image and analyze

In [3]:
image = cv2.imread('shapes_and_colors.png')
resized = imutils.resize(image, width=300)
#get the proportion of the image height and the resized image shapeto get the ratio
ratio = image.shape[0]/float(resized.shape[0])

In [4]:
#convert the resized image to grayscale, blur it (this makes it easier finding edges)
# and treshold
gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5,5), 0)
thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY)[1]

In [5]:
#Find contours in the thresholded image and initialize the shape detector
#cv2.CHAIN_APPROX_SIMPLE is used to get only the corner poiunts. is faster
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
sd = ShapeDetector() #initialize the shapedetector

In [6]:
cnts

(array([[[112, 198]],
 
        [[111, 199]],
 
        [[111, 200]],
 
        [[110, 201]],
 
        [[110, 202]],
 
        [[109, 203]],
 
        [[109, 204]],
 
        [[108, 205]],
 
        [[108, 207]],
 
        [[107, 208]],
 
        [[107, 209]],
 
        [[106, 210]],
 
        [[106, 211]],
 
        [[105, 212]],
 
        [[105, 216]],
 
        [[106, 217]],
 
        [[107, 217]],
 
        [[109, 219]],
 
        [[110, 219]],
 
        [[112, 221]],
 
        [[113, 221]],
 
        [[116, 224]],
 
        [[117, 224]],
 
        [[118, 225]],
 
        [[119, 224]],
 
        [[120, 224]],
 
        [[121, 223]],
 
        [[122, 223]],
 
        [[123, 222]],
 
        [[124, 222]],
 
        [[125, 221]],
 
        [[126, 221]],
 
        [[128, 219]],
 
        [[129, 219]],
 
        [[130, 218]],
 
        [[131, 218]],
 
        [[134, 215]],
 
        [[134, 214]],
 
        [[133, 213]],
 
        [[133, 211]],
 
        [[132, 210]],
 
        [[132, 2

In [7]:
for c in cnts:
    #compute the center of the contour, then detect the name if the shape using 
    #that contour
    M = cv2.moments(c) #find the center of a blob. Weighted average of image pixel intensities
    cX = int((M["m10"] / M["m00"])*ratio) # select the coordinates we need fo both x and y
    cY = int((M["m01"]/ M["m00"]) * ratio)
    shape = sd.detect(c)

    #Multiply the contour (x, y) coordinates by the resize ratio. 
    #Draw these contours and the name of the shape

    c = c.astype("float")
    c *= ratio
    c = c.astype("int")
    #on cv2.drawContours() [c] are the contours and -1 are the indexes argument. In this case it tell the function
    #to draw all contour points
    cv2.drawContours(image, [c], -1, (0,255,0), 2)
    cv2.putText(image, shape, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255,  255), 2)
    
    cv2.imshow("image", image)
    cv2.waitKey(0)

cv2.destroyAllWindows()
    